# Using the pluto MutualFundsIndia data-set

In [ ]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")

#initialize
mfi <- MutualFundsIndia()

### who are the biggest 5 mutual fund managers?

In [ ]:
lastAumDate <- (mfi$AumFundwise() %>%
    summarize(MAX_TS = max(PERIOD)) %>%
    collect())$MAX_TS[1]

mfi$AumFundwise() %>%
    filter(PERIOD == lastAumDate) %>%
    select(FUND) %>%
    mutate(AUM = AVG_AUM_WO_FOFD + AVG_AUM_FOFD) %>%
    arrange(desc(AUM)) %>%
    print(n=5)


### what are the largest 10 mid-cap funds by AUM?

In [ ]:
# Meta to filter for the 'Mid-Cap' category
# AumSchemewise to sort funds by AUM - largest first
# NavTimeSeries to get the start and end dates for which NAVs are available

lastMetaDate <- (mfi$Meta() %>%
    summarize(MAX_TS = max(AS_OF)) %>%
    collect())$MAX_TS[1]

lastSwDate <- (mfi$AumSchemewise() %>%
    summarize(MAX_TS = max(PERIOD)) %>%
    collect())$MAX_TS[1]

print(lastMetaDate)
print(lastSwDate)

mfi$Meta %>%
    join(mfi$AumSchemewise, by="SCHEME_CODE") %>%
    join(mfi$NavTimeSeries, by="SCHEME_CODE") %>%
    group_by(SCHEME_CODE, SCHEME_NAME, CATEGORY, AVG_AUM_WO_FOFD) %>%
    filter(PERIOD == lastSwDate & AS_OF == lastMetaDate & CATEGORY == 'Mid-Cap') %>%
    summarize(start_dt = min(TIME_STAMP), end_dt = max(TIME_STAMP))
    arrange(desc(AVG_AUM_WO_FOFD)) %>%
    print(n=10)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)